# Working with GroupBy

In [1]:
import pandas as pd
import numpy as np

In [2]:
df=pd.DataFrame({"key":list("ABC")*2,
                 "data1":range(6),
                 "data2":np.arange(5,11)})

In [3]:
df

key  data1  data2
0   A      0      5
1   B      1      6
2   C      2      7
3   A      3      8
4   B      4      9
5   C      5     10

In [4]:
group=df.groupby("key")

In [5]:
group.aggregate(["min",np.median,"max"])

data1            data2           
      min median max   min median max
key                                  
A       0    1.5   3     5    6.5   8
B       1    2.5   4     6    7.5   9
C       2    3.5   5     7    8.5  10

In [6]:
group.agg({"data1":"min","data2":"max"})

data1  data2
key              
A        0      8
B        1      9
C        2     10

In [7]:
def f(x):
    return x.max()-x.min()

In [8]:
group.agg(f)

data1  data2
key              
A        3      3
B        3      3
C        3      3

## Applying more than one function

In [9]:
data=pd.DataFrame({"letter":list("ABC")*4,
                   "num":["one","two"]*6,
                   "d1":np.random.randn(12),
                   "d2":np.arange(10,33,2)})

In [10]:
data

letter  num        d1  d2
0       A  one  0.988512  10
1       B  two  1.318566  12
2       C  one  0.606850  14
3       A  two -1.029892  16
4       B  one  0.232158  18
5       C  two  0.380758  20
6       A  one -1.202409  22
7       B  two  0.209867  24
8       C  one -1.164496  26
9       A  two  0.194654  28
10      B  one  0.160770  30
11      C  two -0.301062  32

In [11]:
group=data.groupby(["letter","num"])

In [12]:
group_d1=group["d1"]

In [13]:
group_d1.agg("mean")

letter  num
A       one   -0.106948
        two   -0.417619
B       one    0.196464
        two    0.764216
C       one   -0.278823
        two    0.039848
Name: d1, dtype: float64

In [14]:
group_d1.agg(["mean","std",f])

mean       std         f
letter num                              
A      one -0.106948  1.549215  2.190920
       two -0.417619  0.865885  1.224546
B      one  0.196464  0.050479  0.071388
       two  0.764216  0.783969  1.108699
C      one -0.278823  1.252531  1.771347
       two  0.039848  0.482120  0.681820

In [15]:
group_d1.agg([("f_mean","mean"),
              ("f_std",np.std)])

f_mean     f_std
letter num                    
A      one -0.106948  1.549215
       two -0.417619  0.865885
B      one  0.196464  0.050479
       two  0.764216  0.783969
C      one -0.278823  1.252531
       two  0.039848  0.482120

In [16]:
group.agg({"d1":["count","max","mean"],
           "d2":"sum"})

d1                      d2
           count       max      mean sum
letter num                              
A      one     2  0.988512 -0.106948  32
       two     2  0.194654 -0.417619  44
B      one     2  0.232158  0.196464  48
       two     2  1.318566  0.764216  36
C      one     2  0.606850 -0.278823  40
       two     2  0.380758  0.039848  52

In [17]:
data.groupby(["letter","num"],
             as_index=False).mean()

letter  num        d1  d2
0      A  one -0.106948  16
1      A  two -0.417619  22
2      B  one  0.196464  24
3      B  two  0.764216  18
4      C  one -0.278823  20
5      C  two  0.039848  26

## Split-Apply-Combine

In [18]:
data

letter  num        d1  d2
0       A  one  0.988512  10
1       B  two  1.318566  12
2       C  one  0.606850  14
3       A  two -1.029892  16
4       B  one  0.232158  18
5       C  two  0.380758  20
6       A  one -1.202409  22
7       B  two  0.209867  24
8       C  one -1.164496  26
9       A  two  0.194654  28
10      B  one  0.160770  30
11      C  two -0.301062  32

In [19]:
group=data.groupby("letter")

In [20]:
group["d2"].apply(lambda x:x.describe())

letter       
A       count     4.000000
        mean     19.000000
        std       7.745967
        min      10.000000
        25%      14.500000
        50%      19.000000
        75%      23.500000
        max      28.000000
B       count     4.000000
        mean     21.000000
        std       7.745967
        min      12.000000
        25%      16.500000
        50%      21.000000
        75%      25.500000
        max      30.000000
C       count     4.000000
        mean     23.000000
        std       7.745967
        min      14.000000
        25%      18.500000
        50%      23.000000
        75%      27.500000
        max      32.000000
Name: d2, dtype: float64

In [21]:
math=pd.DataFrame({"Class":list("AB")*3,
                   "Stu":["Kim","Sam",
                          "Tim","Tom","John","Kate"],
                   "Score":[60,70,np.nan,
                            55,np.nan,80]})
math

Class   Stu  Score
0     A   Kim   60.0
1     B   Sam   70.0
2     A   Tim    NaN
3     B   Tom   55.0
4     A  John    NaN
5     B  Kate   80.0

In [22]:
group=math.groupby("Class")

In [23]:
group.mean()

Score
Class           
A      60.000000
B      68.333333

In [24]:
func=lambda f:f.fillna(f.mean())

In [25]:
group.apply(func)

Class   Stu  Score
Class                     
A     0     A   Kim   60.0
      2     A   Tim   60.0
      4     A  John   60.0
B     1     B   Sam   70.0
      3     B   Tom   55.0
      5     B  Kate   80.0

In [26]:
value={"A":100,"B":50}

In [27]:
func1=lambda f:f.fillna(value[f.name])

In [28]:
group.apply(func1)

Class   Stu  Score
0     A   Kim   60.0
1     B   Sam   70.0
2     A   Tim  100.0
3     B   Tom   55.0
4     A  John  100.0
5     B  Kate   80.0